### xgboost

**The most accurate modeling technique for structured data**

In this tutoral, you will learn how to build and optimize models with **gradien boosting**. This method dominates many Kaggle competitions and achieves state-of-the-art results on a variety of datasets.

**Introduction**

For much of this course, you have made predictions with the random forest method, which achieves better preformance than a single decision tree simply by averaging the predictions of many decision trees.

We refer to the random forest method as an "ensemble method".By definition,**ensemble methods** combine the predictions of several models(e.g., several trees, in the case of random forests).

Next, we'll learn about another ensemble method called gradient boosting.

**Gradient Boosting** is a method that goes through cycles to iteratively add models into an ensemble.

It begins by initializing the ensemble with a single model, whose predictions can be pretty native.
(Even if its predictions are wildly inaccurate, subsequent additions to the ensemble will address those errors.)

Then, we start the cycle:
 - First, we use the current ensemble to generate predictions for each observation in the dataset. To make a prediction, we add the predictions from all models in the ensemble.
 - These predictions are used to calculate a loss function(like [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error), for instance).
 - Then, we use the loss function to fit a new model that will be added to the ensemble. Specifically,we determine model parameters so that adding this new model to the ensemble will reduce the loss.(Side note: The "gradient" in "gradient boosting" refers to the fact that we'll use [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) on the loss function to determine the paramenters in this new model.)
 - Finally, we add the new model to ensemble, and...
 - ...repeat!

![示意图](MvCGENh.png)

### Example

**ONE**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In this example, you'll work with the XGBoost library. **XGBoost** stands for extreme gradient boosting, which is an implementation of gradient boosting with several additional features focused on performance and speed.(*Scikit-learn has another version of gradient boosting, but XGBoost has some technical advantages.*)

In the next code cell, we import the scikit-learn API for XGBoost(`xgbhoost, XGBRegressor`).This allows us to bulid and fit a model just as we would in scikit-learn. As you'll see in the output, the `XGBRegressor` class has many tunable paramenters -- you'll learn about those soon!

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

We also make predictions and evaluate the model

In [ ]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

### Parameter Tuning

**n_estimators**

`n_estimators` specifies how many times to go through the modeling cycle described above. It is equal to the number of models that we include in the ensemble.

 - Too low a value causes *underfitting*, which leads to inaccurate predictions on both training data and test data.
 - Too high a value causes *overfitting*, which leads to inaccurate predictions on training data, but inaccurate predictions on test data(*which is what we care about*).
 
Typical values range from 100-1000, through this depends a lot on the `learning_rate` parameter discussed below.


In [ ]:
my_model = XGBRegressor(n_estimators = 500)
my_model.fit(X_train, y_train)

### early_stoping_rounds

`early_stopping_rounds` offers a way to automatically find the ideal value for `n_estimators`.

When using `early_stopping_rounds`, you also need to set aside some data for calculating the validation scores - this is done by setting the `eval_set` parameter.

In [ ]:
my_model = XGBRegressor(n_estimators = 500)
my_model.fit(X_train, y_train,
            early_stopping_rounds= 5,
            eval_set= [(X_valid, y_valid)],
            verbose= False)
#verbose :罗唆,冗长的,累赘的

### learning_rate

In [ ]:
my_model = XGBRegressor(n_estimators= 1000, learning_rate= 0.05)
my_model.fit(X_train, y_train,
            early_stopping_rounds= 5,
            eval_set= [(X_valid, y_valid)],
            verbose= False)

#### n_jobs

In [ ]:
my_model = XGBRegressor(n_estimators= 1000, learning_rate= 0.05, n_jobs= 4)
my_model.fit(X_train, y_train,
            early_stopping_rounds= 5,
            eval_set= [(X_valid, y_valid)],
            verbose= False)

### Conclusion

[XGBoost](https://xgboost.readthedocs.io/en/latest/) is a the leading software library for working with standard tabular data(the type of data you store in Pandas DataFrames, as opposed to more exotic types of data like images and videos). With careful parameter tuning, you can train highly accurate models.